# Modelos CNN

# Database

In [187]:
import os
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from tqdm import tqdm
from improved_model import ImprovedCNNModel as  ElVostreModel # Assuming you have an improved model defined in this module
# Paralelize the model
import multiprocessing
# Learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [188]:
start_time = time.time()

In [189]:
# Añade este código después de cargar el dataset pero antes de crear el dataloader
def undersample_dataset(dataset, random_seed=123):
    # Asegurar reproducibilidad
    random.seed(random_seed)
    
    # Contar muestras por clase
    class_indices = {}
    for idx, (_, label) in enumerate(dataset.samples):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)
    
    # Encontrar la clase con menos muestras
    min_class_count = min([len(indices) for indices in class_indices.values()])
    print(f"Clase minoritaria: {min_class_count} muestras")
    
    # Seleccionar muestras para el dataset balanceado
    balanced_indices = []
    for label, indices in class_indices.items():
        # Seleccionar aleatoriamente min_class_count muestras
        selected = random.sample(indices, min_class_count)
        balanced_indices.extend(selected)
    
    print(f"Dataset original: {len(dataset)} muestras → Dataset balanceado: {len(balanced_indices)} muestras")
    
    # Crear un subconjunto del dataset
    return Subset(dataset, balanced_indices)


In [190]:
torch.manual_seed(123)
random.seed(123)
np.random.seed(123)


# ---------- Dataset ----------
# Transforma para entrenamiento (con data augmentation)
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) 
])

# Transforma para validación (sin augmentation)
val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

data_dir = "data/train"
dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

class_names = dataset.classes
num_classes = len(class_names)

# Aplicar balanceo SOLO al dataset de entrenamiento
# dataset = undersample_dataset(dataset)

# ---------- Validació ----------
val_dir = "data/validation"
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)



In [191]:
lr = 0.01
batch_size = 64
max_total_time = 1800 
num_core = multiprocessing.cpu_count()
weight_decay = 1e-4
dataloader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        shuffle=True, 
                        num_workers=num_core-1,
                        persistent_workers=True,
                        prefetch_factor=4)
val_loader = DataLoader(val_dataset, 
                        batch_size=batch_size, 
                        shuffle=False, 
                        num_workers=num_core-1,
                        persistent_workers=True,
                        prefetch_factor=4)


In [192]:
# ---------- Model ----------
class ElVostreModel(nn.Module):
    def __init__(self, num_classes):
        super(ElVostreModel, self).__init__()
        self.net = nn.Sequential(
            # Bloque 1: 28×28 → 14×14
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Bloque 2: 14×14 → 7×7
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Bloque 3: 7×7 → 3×3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256), 
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [193]:
device = torch.device("cpu")
model = ElVostreModel(num_classes).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
# Añadir el scheduler
scheduler = ReduceLROnPlateau(optimizer, 
                              mode='max', 
                              factor=0.1, 
                              patience=1,
                              threshold=0.01
                              )

criterion = nn.CrossEntropyLoss()

In [195]:
def evaluate(model, loader, name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [196]:
epoch = 0

best_val_acc = 0.0
model_save_path = "best_model.pth"

while True:
    start_epoch = time.time()
    epoch += 1
    model.train()
    loop = tqdm(dataloader, desc=f"Època {epoch}", leave=False)

    for i, (images, labels) in enumerate(loop):
        if time.time() - start_time > max_total_time-60:
            print("Temps màxim assolit. Fi de l'entrenament")
            break

        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())
    if time.time() - start_time > max_total_time-60:
        break
    # Validació per època
    val_acc = evaluate(model, val_loader, f"Validació (després de la època {epoch}[{time.time() - start_epoch:.2f}s])")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_save_path)
        print(f"Nou millor model guardat amb {best_val_acc * 100:.2f}% de precisió de validació")
    scheduler.step(val_acc)  # Ajusta el LR basado en la precisión de validación

Validació (després de la època 1[152.14s]) Accuracy: 60.87%
Nou millor model guardat amb 60.87% de precisió de validació


Validació (després de la època 2[109.23s]) Accuracy: 59.42%


Validació (després de la època 3[102.21s]) Accuracy: 63.77%
Nou millor model guardat amb 63.77% de precisió de validació


Validació (després de la època 4[101.08s]) Accuracy: 57.25%


Validació (després de la època 5[101.03s]) Accuracy: 64.49%
Nou millor model guardat amb 64.49% de precisió de validació


...

Validació (després de la època 13[100.40s]) Accuracy: 67.39%


Validació (després de la època 14[98.03s]) Accuracy: 68.12%


Validació (després de la època 15[99.86s]) Accuracy: 68.12%


Validació (després de la època 16[106.37s]) Accuracy: 68.12%


Temps màxim assolit. Fi de l'entrenament


# Final Avaluacio

In [197]:
def load_best_model(path, num_classes):
    """Carga el mejor modelo guardado."""
    model = ElVostreModel(num_classes)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [198]:
best_model = load_best_model(model_save_path, num_classes)
# Evaluar el modelo en el conjunto de entrenamiento y validación
train_acc = evaluate(best_model,dataloader, "Train (subset)")
val_acc = evaluate(best_model, val_loader, "Validation (final)")

print(f"\nFinal Metrics:")
print(f"   Train Accuracy: {train_acc * 100:.2f}%")
print(f"   Validation Accuracy: {val_acc * 100:.2f}%")
print(f"Temps total d'entrenament: {time.time() - start_time:.2f} segons")

Train (subset) Accuracy: 80.38%
Validation (final) Accuracy: 68.84%

Final Metrics:
   Train Accuracy: 80.38%
   Validation Accuracy: 68.84%
Temps total d'entrenament: 1778.06 segons


## Afegir els pessos